# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
!jupyter nbextension list
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension# enable gmaps
!jupyter nbextension enable --py --sys-prefix gmaps# install gmaps - double check if already installed
!pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

Known nbextensions:
  config dir: C:\Users\JohnB\OneDrive\Documents\Anaconda\etc\jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 
      jupyter-gmaps/extension enabled 


      - Validating: ok
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Please specify one nbextension/package at a time
Please specify one nbextension/package at a time


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
weather.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = weather["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowdown_city_df = weather.loc[(weather["Max Temp"] > 70) & (weather["Max Temp"] < 80) & (weather["Cloudiness"] == 0), :]
narrowdown_city_df = narrowdown_city_df.dropna(how='any')
narrowdown_city_df.reset_index(inplace=True)
del narrowdown_city_df['index']
narrowdown_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
5,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
6,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
7,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
9,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotellist = []

for i in range(len(narrowdown_city_df)):
    lat = narrowdown_city_df.loc[i]['Lat']
    lng = narrowdown_city_df.loc[i]['Lng']

    params = { "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")



In [7]:
narrowdown_city_df["Hotel Name"] = hotellist
narrowdown_city_df = narrowdown_city_df.dropna(how='any')
narrowdown_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Pisco
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Koumac
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
5,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
6,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador
7,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mogok
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,Namibe
9,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Birjand


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowdown_city_df.iterrows()]
locations = narrowdown_city_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))